In [1]:
!cat /proc/cpuinfo | grep "model name" | head -1

model name	: Intel(R) Xeon(R) CPU E5-2695 v4 @ 2.10GHz


# Arithmetic Units 

In [2]:
import numpy as np
import d2ltvm
import tvm
from tvm import te

successd...



In [3]:
def benchmark(dtype):
    x = np.random.normal(size=(1000, 1000)).astype(dtype)
    %timeit np.dot(x, x)

benchmark('float32')
benchmark('float64')

benchmark('int32')
benchmark('int64')

4 ms ± 493 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.95 ms ± 476 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.09 s ± 21.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.09 s ± 17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
x = np.random.normal(size=(1000, 1000)).astype('float32')
%timeit np.exp(x)

1.61 ms ± 27.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Parallel Exection

In [5]:
!lscpu | grep MHz

CPU MHz:             1200.130
CPU max MHz:         3300.0000
CPU min MHz:         1200.0000


In [28]:
!cat /proc/cpuinfo | grep "flags" | head -1
# SIMD

flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf pni pclmulqdq dtes64 monitor ds_cpl vmx smx est tm2 ssse3 sdbg fma cx16 xtpr pdcm pcid dca sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand lahf_lm abm 3dnowprefetch cpuid_fault epb cat_l3 cdp_l3 invpcid_single pti intel_ppin ssbd ibrs ibpb stibp tpr_shadow vnmi flexpriority ept vpid fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm cqm rdt_a rdseed adx smap intel_pt xsaveopt cqm_llc cqm_occup_llc cqm_mbm_total cqm_mbm_local dtherm ida arat pln pts md_clear flush_l1d
grep: write error: Broken pipe
cat: write error: Broken pipe


In [29]:
!cat /proc/cpuinfo | grep "model name"  | wc -l
# multi-cores

36


In [30]:
''' performance
    physical_cores * #cycles_per_second * #instructions_per_cycle * #operations_per_instruction
    36(multi-cores) * 1.2GHz * 8(SIMD:AVX2 256 / 32) * 2 FMA
'''
36 * 1.2 * 8 * 2

691.1999999999999

In [31]:
x = np.random.normal(size=(1000, 1000)).astype('float32')
res = %timeit -o -q np.dot(x, x)
2 * 1000 ** 3 / res.average / 1e9

503.06367183652367

# Memory Subsystem

In [32]:
!cat /proc/meminfo | grep MemTotal

MemTotal:       131911492 kB


In [34]:
# memory bandwidth
# eg. 640 * 10e9 ops/sec; memory bandwidth 640 * 10e9 * 4 B/sec

In [35]:
!lscpu | grep cache

L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            46080K


# Summary
1.CPUs have dedicated units to handle computations on various data types. A CPU’s peak performance is determined by the clock rate, the number of cores, and the instruction sets. \
2.CPUs use multi-level caches to bridge the gap between CPU computational power and main memory bandwidth. \
3.An efficient program should be effectively parallelized and access data with good temporal and spatial localities.